# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das Linhas de Ônibus
Neste notebook, realizarei a extração dos dados das linhas de ônibus em Limeira, SP. Coletarei as localizações dos pontos de cada linha e a distância de cada ponto da faculdade.

In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from geopy.geocoders import Nominatim
import geopy
import pandas as pd

In [2]:
url = 'https://moovitapp.com/index/pt-br/transporte_p%C3%BAblico-line-02_OLGA_VERONI_X_GEADA-Limeira-4161-908539-743590-0'
response = urlopen(url)
html_linhas_onibus = response.read()
soup_linhas_onibus = BeautifulSoup(html_linhas_onibus, 'html.parser')

In [3]:
pontos = []

for ponto in soup_linhas_onibus.find('ul', {'class': 'stops-list bordered'}).find_all('h3'):
  pontos.append(ponto.get_text())

In [4]:
pontos = pd.Series(pontos)
pontos = pontos.unique()
pontos

array(['Ponto Final Geada', 'Rua Elza Hergert Marostegan',
       'Avenida Prefeito Ari Levi Pereira, 368-408',
       'Avenida Canadá, 456-504',
       'Avenida Estados Unidos Da América, 52-100',
       'Rua Capitão Joaquim Manoel Pereira, 333-373',
       'Avenida Doutor João Amaral Gama, 575-633',
       'Avenida Doutor João Amaral Gama, 391 (Rapidão)',
       'Rua Júlio Wenzel, 641-655',
       'Rua Professor Rui Côrte Brilho, 816-880',
       'Rua Oscár Vargas, 17-87', 'Rua Oswaldo Bertolini, 505',
       'Praça Primeiro De Maio, 112-146',
       'Avenida Cônego Manoel Alves, 775',
       'Norte Avenida Doutor Fabrício Vampré, 975-991',
       'Norte Avenida Doutor Fabrício Vampré, 685',
       'Norte Avenida Doutor Fabrício Vampré, 131-181',
       'Rua Capitão Francisco Sérgio De Toledo, 145-207', 'Nosso Clube',
       'Medical B/C', 'Rua Barão De Campinas, 1027',
       'Escola Brasil - Praça José Bonifácio B/C',
       'Rua Barão De Campinas, 301', 'Terminal Central De Limeir

In [9]:
lat = []
lon = []

geolocator = Nominatim(user_agent="EstadiaLimeiraApp")

for ponto in pontos:
  location = geolocator.geocode(f"{ponto}, Limeira, Brazil")
  if location == None:
    coords = input(f"Digite a coordenada de '{ponto}' (x, y): ")
    coords = coords.split(',')
    coords = (float(coords[0]), float(coords[1]))
    lat.append(coords[0])
    lon.append(coords[1])
  else:
    lat.append(location.latitude)
    lon.append(location.longitude)

Digite a coordenada de 'Ponto Final Geada' (x, y): -22.54534799380873, -47.45448054085058
Digite a coordenada de 'Avenida Doutor João Amaral Gama, 391 (Rapidão)' (x, y): -22.558555444439218, -47.442630355315096
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 816-880' (x, y): -22.561134208423862, -47.43595336973164
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 975-991' (x, y): -22.559066875118823, -47.4229054440875
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 685' (x, y): -22.558620803921194, -47.42013176091486
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 131-181' (x, y): -22.560728231386136, -47.41555660069769
Digite a coordenada de 'Rua Capitão Francisco Sérgio De Toledo, 145-207' (x, y): -22.558197007552785, -47.412828403187596
Digite a coordenada de 'Nosso Clube' (x, y): -22.558720436605455, -47.409808033403905
Digite a coordenada de 'Medical B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Escola 

KeyboardInterrupt: ignored

In [ ]:
'''
1. -22.54534799380873, -47.45448054085058
2. -22.558555444439218, -47.442630355315096
3. -22.561134208423862, -47.43595336973164
4. -22.559066875118823, -47.4229054440875
5. -22.558620803921194, -47.42013176091486
6. -22.560728231386136, -47.41555660069769
7. -22.558197007552785, -47.412828403187596
8. -22.558720436605455, -47.409808033403905
9. -22.561011950736074, -47.40396744752847
10. -22.569341176363963, -47.400102804456175
11. -22.577078622765704, -47.39148226761585
12. -22.581854394170282, -47.38515237650475
13. -22.579961253882317, -47.381879508771206
14. -22.581196104338872, -47.36869089689841
'''

In [ ]:
df_ponto_02 = pd.DataFrame()
df_ponto_02['ponto'] = pontos
df_ponto_02['lat'] = lat
df_ponto_02['lon'] = lon

df_ponto_02.head()

In [ ]:
print(f'Foram encontrados {len(df_ponto_02)} pontos de ônibus')

In [ ]:
import folium
from pprint import pprint

# Criação do mapa
map = folium.Map(location=[-22.56508562812944, -47.40434392781407], tiles='cartodbpositron', zoom_start=20)

# Tooltip
tooltip_acomodacao = "Clique para ver o nome do ponto"

# Markers
for i in range(len(df_ponto_02)):
  folium.Marker(df_ponto_02.iloc[i, 1:], popup=f"<i>Ponto {i+1}: {df_ponto_02.iloc[i, 0]}</i>", tooltip=tooltip_acomodacao,icon=folium.Icon(color="blue", icon="bus", prefix='fa')).add_to(map)

map